In [38]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [41]:
keywords = pd.read_csv('keywords_preprocessed.csv')
meta = pd.read_csv('movies_metadata_preprocessed.csv')
ratings = pd.read_csv('ratings_preprocessed.csv')

meta.drop(["belongs_to_collection"],axis=1)
meta
#75-25 Split of Data
# train,test = train_test_split(meta,test_size=0.25)

# regr = linear_model.LinearRegression()
# regr.fit(meta,meta.revenue)

,Unnamed: 0,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,keywords
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,['Pixar Animation Studios'],1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,"['jealousy', 'toy', 'boy', 'friendship', 'frie..."
1,1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,"['TriStar Pictures', 'Teitler Film', 'Intersco...",1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"['board game', 'disappearance', ""based on chil..."
2,2,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,['Twentieth Century Fox Film Corporation'],1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"['based on novel', 'interracial relationship',..."
3,3,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,"['Sandollar Productions', 'Touchstone Pictures']",1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"['baby', 'midlife crisis', 'confidence', 'agin..."
4,4,False,NaN,60000000,"['Action', 'Crime', 'Drama', 'Thriller']",949,tt0113277,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,"['Regency Enterprises', 'Forward Pass', 'Warne...",1995-12-15,187436818.0,170.0,Released,A Los Angeles Crime Saga,Heat,7.7,1886.0,"['robbery', 'detective', 'bank', 'obsession', ..."
5,5,False,NaN,35000000,"['Action', 'Adventure', 'Thriller']",9091,tt0114576,en,International action superstar Jean Claude Van...,5.231580,"['Universal Pictures', 'Imperial Entertainment...",1995-12-22,64350171.0,106.0,Released,Terror goes into overtime.,Sudden Death,5.5,174.0,"['terrorist', 'hostage', 'explosive', 'vice pr..."
6,6,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"['Adventure', 'Action', 'Thriller']",710,tt0113189,en,James Bond must unmask the mysterious head of ...,14.686036,"['United Artists', 'Eon Productions']",1995-11-16,352194034.0,130.0,Released,No limits. No fears. No substitutes.,GoldenEye,6.6,1194.0,"['cuba', 'falsely accused', 'secret identity',..."
7,7,False,NaN,62000000,"['Comedy', 'Drama', 'Romance']",9087,tt0112346,en,"Widowed U.S. president Andrew Shepherd, one of...",6.318445,"['Columbia Pictures', 'Castle Rock Entertainme...",1995-11-17,107879496.0,106.0,Released,Why can't the most powerful man in the world h...,The American President,6.5,199.0,"['white house', 'usa president', 'new love', '..."
8,8,False,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"['Family', 'Animation', 'Adventure']",21032,tt0112453,en,An outcast half-wolf risks his life to prevent...,12.140733,"['Universal Pictures', 'Amblin Entertainment',...",1995-12-22,11348324.0,78.0,Released,Part Dog. Part Wolf. All Hero.,Balto,7.1,423.0,"['wolf', 'dog-sledding race', 'alaska', 'dog',..."
9,9,False,NaN,44000000,"['History', 'Drama']",10858,tt0113987,en,An all-star cast powers this epic look at Amer...,5.092000,"['Hollywood Pictures', 'Cinergi Pictures Enter...",1995-12-22,13681765.0,192.0,Released,"Triumphant in Victory, Bitter in Defeat. He Ch...",Nixon,7.1,72.0,"['usa president', 'presidential election', 'wa..."
